In [1]:
from pathlib import Path 
import os, dotenv
dotenv.load_dotenv()
os.chdir(Path(os.getenv("PYTHONPATH")).expanduser())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from src.preprocessing.quality_process import compute_file_metrics

In [3]:
# Source code location
# /Users/josh/SecurityAnalytics/development/cryptol
# /Users/josh/SecurityAnalytics/development/cryptol-specs
# /Users/josh/SecurityAnalytics/development/saw-script
NEW_VERSION = "1.2s"
VARIANTION = "nocomments"
# --- dataset ---
OLD_DATA = f"verified_{VARIANTION}.jsonl"  # e.g., verified_nocomments
# --- paths ---
df = pd.read_json(f"data/training_datasets/{OLD_DATA}", lines=True)
df.head()


,filename,filetype,content,variant
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cry,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cry,module AES128 where\n\nimport `Common::AES\nim...,without_comments
2,AES-GCM-SIV-proof/proof/cryptol-specs/intrinsi...,cry,module Intrinsics where\n\nimport `Common::AES...,without_comments
3,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cry,module AES256 where\n\nimport `Common::AES\nim...,without_comments
4,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cry,type Nb = 4\ntype State = [4][Nb]...,without_comments


In [4]:
with open("syntax_verification_failures.txt", "r") as f:
    files_to_remove = {line.strip() for line in f if line.strip()}

# 2. Drop rows where df["filename"] is in that list
df_filtered = df[~df["filename"].isin(files_to_remove)].reset_index(drop=True).copy()

# optional: if you want to modify df in-place
df = df_filtered

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  424 non-null    object
 1   filetype  424 non-null    object
 2   content   424 non-null    object
 3   variant   424 non-null    object
dtypes: object(4)
memory usage: 13.4+ KB


In [5]:
df.to_json(f"data/training_datasets/verified_{VARIANTION}_{NEW_VERSION}.jsonl", orient='records', lines=True)